In [1]:
#pip install -r requirements.txt
#(needed libraries are installed with version because of langchain may difficult when version error)


In [2]:
import os
os.environ["GOOGLE_API_KEY"] = (gemini api key)
# gemini api key uploaded


In [3]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Udhayakumar B Resume.pdf")
documents = loader.load()
#my resume 2 pages uploaded in pdf format


C:\Users\udhaya\anaconda3\envs\agenticai\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
docs = splitter.split_documents(documents)
#loaded 2 page resume splitted into varuious chunks due to easy retrieval


In [5]:
docs

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-24T16:17:17+00:00', 'title': 'Udhayakumar resume.pdf', 'moddate': '2025-08-24T16:17:16+00:00', 'keywords': 'DAGxAHs2rCY,BAFl5an3MA8,0', 'author': 'UDHAYA KUMAR', 'source': 'Udhayakumar B Resume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Github +91-8870172428\nMayiladuthurai,609811\n udhayaaiengineer@ gmail.comExperienced professional with 10+ years in mechanical\noperations specializing in maintenance coordination,\ndocumentation, and resource management. Recently upskilled\nin AI and Data Science, with expertise in data analysis,\nvisualization, and machine learning. Known for improving\nefficiency, ensuring safety, and delivering projects on time.\nReady to combine mechanical expertise and AI skills to drive\ninnovation and business success.\nDeveloped dashboards and reports to track performance and'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationda

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Use HuggingFace embeddings (free, local)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Store in Chroma
vectorstore = Chroma.from_documents(
    docs, 
    embeddings, 
    persist_directory="db"
)

retriever = vectorstore.as_retriever()
#chunks of text data are converted into  meaningful numerical number for easy understand purposes 
#here huggingface embedding model used and vectore store chromadb used


C:\Users\udhaya\AppData\Local\Temp\ipykernel_2376\2067003000.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [7]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001BCED786C90>, search_kwargs={})

In [8]:
#!pip install --upgrade langchain langchain-core langchain-community

In [9]:
# 2. Ensure Pydantic is on V2
#!pip install "pydantic>=2.0"

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Use the official, current model name
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.2) # Or "gemini-2.5-pro"

In [11]:
#  Conversational RAG chain
from langchain.chains import ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

In [12]:
#  Memory for chat history
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\udhaya\AppData\Local\Temp\ipykernel_2376\2137983393.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [17]:
#  Wrap RAG as a StructuredTool
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
def rag_tool_fn(question: str) -> str:
    return rag_chain.invoke({
        "question": question,
        "chat_history": []
    })["answer"]

rag_tool = StructuredTool.from_function(
    name="RAG_QA",
    description="Useful for answering questions about resume summary from the provided documents.",
    func=rag_tool_fn
)

In [18]:
#  Create agent
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

In [19]:
#  Run conversation
print("1️⃣ First question")
res1 = agent.run("What are the skills used?")
print("Answer:", res1)

print("\n2️⃣ Follow-up")
res2 = agent.run("Who created it?")
print("Answer:", res2)

print("\n3️⃣ Ask again")
res3 = agent.run("Explain and summaries resume simply.")
print("Answer:", res3)


1️⃣ First question


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The skills used include: Time management, Organization, Leadership, Initiative, Technical support, Communication Skills, Looker Studio, SSD, MobileNet, TensorFlow, OpenCV, Python programming, NumPy, Natural Language Processing (NLP), and Generative AI.

> Finished chain.
Answer: The skills used include: Time management, Organization, Leadership, Initiative, Technical support, Communication Skills, Looker Studio, SSD, MobileNet, TensorFlow, OpenCV, Python programming, NumPy, Natural Language Processing (NLP), and Generative AI.

2️⃣ Follow-up


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: The project "Real Time Project Dashboard using Looker Studio" was created by HopeAI Technologies.

> Finished chain.
Answer: The project "Real Time Project Dashboard using Looker Studio" was created by HopeAI Technologies.

3️⃣ Ask again


> Entering new AgentExecutor ch